<a href="https://colab.research.google.com/github/saugata-malakar/ML-CaPsule/blob/master/Almgren_Chriss_Market_Impact_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install streamlit websocket-client numpy pandas scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.1 MB/s eta 0:00:00


In [2]:
import streamlit as st
from websocket_handler import start_websocket

st.set_page_config(layout="wide")

# Left Panel: Inputs
with st.sidebar:
    st.header("Trade Parameters")
    exchange = st.selectbox("Exchange", ["OKX"])
    asset = st.text_input("Spot Asset", "BTC-USDT-SWAP")
    order_type = st.selectbox("Order Type", ["Market"])
    quantity = st.number_input("Quantity (USD)", value=100.0)
    volatility = st.slider("Volatility (%)", 0.1, 10.0, 1.0)
    fee_tier = st.selectbox("Fee Tier", ["Tier 1", "Tier 2", "Tier 3"])

# Right Panel: Outputs
st.header("Simulation Results")
slippage = st.empty()
fees = st.empty()
impact = st.empty()
net_cost = st.empty()
maker_taker_ratio = st.empty()
latency


ModuleNotFoundError: No module named 'websocket_handler'

In [3]:
# Create a placeholder file for websocket_handler.py if it doesn't exist
# You will need to implement the actual start_websocket function later.
# As an example, you can create this file in the same directory as your notebook
# with the following minimal content:

# --- Start of websocket_handler.py content ---
# import websocket
# import threading

# def start_websocket():
#     print("Websocket handler is running (placeholder)")
#     # Implement your websocket logic here
#     # Example using websocket-client:
#     # ws = websocket.WebSocketApp("ws://example.com",
#     #                         on_open=on_open,
#     #                         on_message=on_message,
#     #                         on_error=on_error,
#     #                         on_close=on_close)
#     # ws_thread = threading.Thread(target=ws.run_forever)
#     # ws_thread.daemon = True # Allow the main program to exit even if the thread is running
#     # ws_thread.start()

# def on_open(ws):
#     print("WebSocket opened")

# def on_message(ws, message):
#     print(f"Received: {message}")

# def on_error(ws, error):
#     print(f"Error: {error}")

# def on_close(ws, close_status_code, close_msg):
#     print(f"WebSocket closed with status {close_status_code}: {close_msg}")
# --- End of websocket_handler.py content ---

# Ensure the websocket_handler.py file exists in the same directory as this notebook
# before running this code block.

from IPython import get_ipython
from IPython.display import display
# %%
# Use !pip install to run pip as a shell command
!pip install streamlit websocket-client numpy pandas scikit-learn

# %%
import streamlit as st
# Assuming websocket_handler.py is a local file
from websocket_handler import start_websocket

st.set_page_config(layout="wide")

# Left Panel: Inputs
with st.sidebar:
    st.header("Trade Parameters")
    exchange = st.selectbox("Exchange", ["OKX"])
    asset = st.text_input("Spot Asset", "BTC-USDT-SWAP")
    order_type = st.selectbox("Order Type", ["Market"])
    quantity = st.number_input("Quantity (USD)", value=100.0)
    volatility = st.slider("Volatility (%)", 0.1, 10.0, 1.0)
    fee_tier = st.selectbox("Fee Tier", ["Tier 1", "Tier 2", "Tier 3"])

# Right Panel: Outputs
st.header("Simulation Results")
slippage = st.empty()
fees = st.empty()
impact = st.empty()
net_cost = st.empty()
maker_taker_ratio = st.empty()
latency = st.empty() # Added initialization for 'latency' using st.empty()

# %%
# You might want to call the start_websocket function here or based on some event
# start_websocket()

# %%

ModuleNotFoundError: No module named 'websocket_handler'

In [4]:
import os

file_content = """
import websocket
import threading

def start_websocket():
    print("Websocket handler is running (placeholder)")
    # Implement your websocket logic here
    # Example using websocket-client:
    # ws = websocket.WebSocketApp("ws://example.com",
    #                         on_open=on_open,
    #                         on_message=on_message,
    #                         on_error=on_error,
    #                         on_close=on_close)
    # ws_thread = threading.Thread(target=ws.run_forever)
    # ws_thread.daemon = True # Allow the main program to exit even if the thread is running
    # ws_thread.start()

def on_open(ws):
    print("WebSocket opened")

def on_message(ws, message):
    print(f"Received: {message}")

def on_error(ws, error):
    print(f"Error: {error}")

def on_close(ws, close_status_code, close_msg):
    print(f"WebSocket closed with status {close_status_code}: {close_msg}")
"""

file_name = "websocket_handler.py"

# Check if the file already exists to avoid overwriting
if not os.path.exists(file_name):
    with open(file_name, "w") as f:
        f.write(file_content)
    print(f"Created placeholder file: {file_name}")
else:
    print(f"File '{file_name}' already exists.")

Created placeholder file: websocket_handler.py


In [6]:
# websocket_handler.py

import asyncio
import websockets
import json
import time
from threading import Thread

def start_websocket(symbol="BTC-USDT-SWAP", update_callback=None):
    ws_url = f"wss://ws.gomarket-cpp.goquant.io/ws/l2-orderbook/okx/{symbol}"

    def run():
        async def connect():
            async with websockets.connect(ws_url) as websocket:
                print(f"Connected to WebSocket: {ws_url}")
                while True:
                    start_time = time.time()
                    message = await websocket.recv()
                    data = json.loads(message)

                    latency = time.time() - start_time

                    if update_callback:
                        update_callback(data, latency)

        asyncio.new_event_loop().run_until_complete(connect())

    thread = Thread(target=run)
    thread.start()


In [7]:
# models.py

import numpy as np

def estimate_slippage(order_quantity_usd, orderbook):
    top_bid = float(orderbook["bids"][0][0])
    slippage = 0.001 * np.log1p(order_quantity_usd)  # Dummy logic
    return top_bid * slippage

def estimate_fees(order_quantity_usd, fee_rate=0.001):
    return order_quantity_usd * fee_rate

def estimate_market_impact(order_quantity_usd):
    return 0.002 * np.sqrt(order_quantity_usd)  # Simplified Almgren-Chriss proxy

def predict_maker_taker_proportion():
    return 0.3, 0.7  # Dummy values

def calculate_net_cost(slippage, fees, impact):
    return slippage + fees + impact


In [12]:
# 🟢 COMBINED FILE: main.py (all-in-one)

import asyncio
import websockets
import json
import time
from threading import Thread
import numpy as np
import streamlit as st

# WebSocket Handler
def start_websocket(symbol="BTC-USDT-SWAP", update_callback=None):
    ws_url = f"wss://ws.gomarket-cpp.goquant.io/ws/l2-orderbook/okx/{symbol}"

    def run():
        async def connect():
            async with websockets.connect(ws_url) as websocket:
                print(f"Connected to WebSocket: {ws_url}")
                while True:
                    start_time = time.time()
                    message = await websocket.recv()
                    data = json.loads(message)
                    latency = time.time() - start_time
                    if update_callback:
                        update_callback(data, latency)
        asyncio.new_event_loop().run_until_complete(connect())

    Thread(target=run).start()

# Models
def estimate_slippage(order_quantity_usd, orderbook):
    top_bid = float(orderbook["bids"][0][0])
    slippage = 0.001 * np.log1p(order_quantity_usd)
    return top_bid * slippage

def estimate_fees(order_quantity_usd, fee_rate=0.001):
    return order_quantity_usd * fee_rate

def estimate_market_impact(order_quantity_usd):
    return 0.002 * np.sqrt(order_quantity_usd)

def predict_maker_taker_proportion():
    return 0.3, 0.7

def calculate_net_cost(slippage, fees, impact):
    return slippage + fees + impact

# Streamlit UI
st.set_page_config(layout="wide")
st.title("📈 GoQuant Real-Time Trade Simulator")

# Sidebar Inputs
with st.sidebar:
    st.header("Trade Parameters")
    asset = st.text_input("Spot Asset Symbol (e.g., BTC-USDT-SWAP)", value="BTC-USDT-SWAP")
    order_type = st.selectbox("Order Type", ["Market"])
    quantity_usd = st.number_input("Order Size (USD)", min_value=10.0, value=100.0)
    volatility = st.slider("Market Volatility", 0.0, 1.0, 0.2)
    fee_tier = st.selectbox("Fee Tier", ["Tier 1", "Tier 2", "Tier 3"])

st.divider()
col1, col2 = st.columns(2)

with col2:
    st.subheader("📊 Output Parameters")
    slippage_out = st.empty()
    fee_out = st.empty()
    impact_out = st.empty()
    net_cost_out = st.empty()
    proportion_out = st.empty()
    latency_out = st.empty()

# Callback to update Streamlit
def update_ui(data, latency):
    slippage = estimate_slippage(quantity_usd, data)
    fees = estimate_fees(quantity_usd)
    impact = estimate_market_impact(quantity_usd)
    net_cost = calculate_net_cost(slippage, fees, impact)
    maker_p, taker_p = predict_maker_taker_proportion()

    slippage_out.metric("Expected Slippage (USD)", f"${slippage:.4f}")
    fee_out.metric("Expected Fees (USD)", f"${fees:.4f}")
    impact_out.metric("Market Impact (USD)", f"${impact:.4f}")
    net_cost_out.metric("Net Cost (USD)", f"${net_cost:.4f}")
    proportion_out.metric("Maker / Taker %", f"{maker_p*100:.1f}% / {taker_p*100:.1f}%")
    latency_out.metric("Internal Latency (ms)", f"{latency*1000:.2f} ms")

# Start WebSocket
start_websocket(symbol=asset, update_callback=update_ui)


2025-05-16 10:40:08.426 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 10:40:08.427 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 10:40:08.428 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 10:40:08.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 10:40:08.430 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 10:40:08.430 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 10:40:08.431 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 10:40:08.432 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [13]:
# models.py
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
import time

# Dummy regression model (replace with trained models or realistic assumptions)
slippage_model = LinearRegression()
maker_taker_model = LogisticRegression()

def estimate_slippage(order_book, quantity_usd):
    bids = order_book["bids"]
    asks = order_book["asks"]

    if not bids or not asks:
        return 0.0

    best_bid = float(bids[0][0])
    best_ask = float(asks[0][0])
    mid_price = (best_bid + best_ask) / 2

    simulated_exec_price = best_ask + (0.0002 * mid_price)  # simplistic slippage approximation
    slippage = simulated_exec_price - mid_price
    return slippage

def estimate_fees(quantity_usd, fee_rate=0.001):
    return quantity_usd * fee_rate

def estimate_market_impact(quantity_usd, volatility=0.02, liquidity=1000000):
    impact = volatility * (quantity_usd / liquidity)
    return impact

def predict_maker_taker_prob(order_type):
    if order_type == 'market':
        return 0.0, 1.0
    else:
        return 1.0, 0.0

def measure_latency(start_time):
    return time.time() - start_time


In [14]:
# websocket_handler.py
import asyncio
import websockets
import json
import threading
import time

def run_websocket(symbol, update_callback):
    async def handler():
        uri = f"wss://ws.gomarket-cpp.goquant.io/ws/l2-orderbook/okx/{symbol}"
        async with websockets.connect(uri) as websocket:
            while True:
                start = time.time()
                message = await websocket.recv()
                data = json.loads(message)
                latency = time.time() - start

                order_book = {
                    "bids": data.get("bids", []),
                    "asks": data.get("asks", [])
                }

                update_callback(order_book, latency)

    def start():
        asyncio.run(handler())

    thread = threading.Thread(target=start)
    thread.daemon = True
    thread.start()


In [16]:
import asyncio
import websockets
import json
import threading
import time

def run_websocket(symbol, update_callback):
    async def handler():
        uri = f"wss://ws.gomarket-cpp.goquant.io/ws/l2-orderbook/okx/{symbol}"
        async with websockets.connect(uri) as websocket:
            while True:
                start = time.time()
                message = await websocket.recv()
                data = json.loads(message)
                latency = time.time() - start

                order_book = {
                    "bids": data.get("bids", []),
                    "asks": data.get("asks", [])
                }

                update_callback(order_book, latency)

    def start():
        asyncio.run(handler())

    thread = threading.Thread(target=start)
    thread.daemon = True
    thread.start()


In [18]:
import numpy as np

def almgren_chriss_impact(order_size, volatility, liquidity, risk_aversion, time_horizon):
    """
    Simplified Almgren-Chriss model to estimate market impact cost.

    Parameters:
    - order_size: Number of units to trade (positive for buy, negative for sell)
    - volatility: Asset volatility (e.g., daily std dev)
    - liquidity: Average daily volume or liquidity measure
    - risk_aversion: Trader’s risk aversion coefficient (higher means more risk-averse)
    - time_horizon: Execution time in hours or days

    Returns:
    - total_cost: Estimated expected cost of market impact
    """

    # Permanent impact coefficient (often linear to order size)
    permanent_impact = 0.1 * abs(order_size) / liquidity

    # Temporary impact coefficient (often scales with sqrt of trading speed)
    trading_speed = abs(order_size) / time_horizon
    temporary_impact = 0.2 * np.sqrt(trading_speed / liquidity)

    # Variance of price movement due to volatility and execution time
    variance = (volatility**2) * time_horizon

    # Expected cost is combination of impact + risk penalty
    expected_cost = permanent_impact + temporary_impact + risk_aversion * variance

    return expected_cost


In [19]:
import time

def measure_latency(func, *args, **kwargs):
    start = time.perf_counter()
    result = func(*args, **kwargs)
    end = time.perf_counter()
    latency_ms = (end - start) * 1000
    return latency_ms, result

# Example inside your websocket handler
async def handler():
    async with websockets.connect(uri) as websocket:
        while True:
            start_receive = time.perf_counter()
            message = await websocket.recv()
            end_receive = time.perf_counter()

            data = json.loads(message)
            receive_latency = (end_receive - start_receive) * 1000

            start_process = time.perf_counter()
            # process data
            update_orderbook(data)
            end_process = time.perf_counter()
            process_latency = (end_process - start_process) * 1000

            # Log or store latencies for benchmarking
            print(f"Receive latency: {receive_latency:.2f} ms, Processing latency: {process_latency:.2f} ms")


In [20]:
# Install websocket-client
!pip install websocket-client

import websocket
import json
import threading
import time

# Global variable to store latest orderbook snapshot
latest_orderbook = {}

def on_message(ws, message):
    global latest_orderbook
    data = json.loads(message)
    latest_orderbook = data
    # Print top bid and ask levels
    print(f"Timestamp: {data.get('timestamp')}")
    print(f"Top Bid: {data['bids'][0]} | Top Ask: {data['asks'][0]}")
    print("-" * 40)

def on_error(ws, error):
    print(f"Error: {error}")

def on_close(ws, close_status_code, close_msg):
    print("### WebSocket closed ###")

def on_open(ws):
    print("WebSocket connection opened")

def run_ws():
    url = "wss://ws.gomarket-cpp.goquant.io/ws/l2-orderbook/okx/BTC-USDT-SWAP"
    ws = websocket.WebSocketApp(url,
                                on_open=on_open,
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close)
    ws.run_forever()

# Start WebSocket in a daemon thread so notebook stays interactive
thread = threading.Thread(target=run_ws)
thread.daemon = True
thread.start()

# Keep notebook alive and print update message
print("WebSocket listener started, listening for 60 seconds...")

for i in range(60):
    time.sleep(1)

print("Finished listening.")


WebSocket listener started, listening for 60 seconds...
WebSocket connection opened
Timestamp: 2025-05-16T10:44:57Z
Top Bid: ['103704.6', '1259.35'] | Top Ask: ['103704.7', '1867.43']
----------------------------------------
Timestamp: 2025-05-16T10:44:57Z
Top Bid: ['103704.6', '1275.35'] | Top Ask: ['103704.7', '1807.72']
----------------------------------------
Timestamp: 2025-05-16T10:44:58Z
Top Bid: ['103704.6', '1275.63'] | Top Ask: ['103704.7', '1807.86']
----------------------------------------
Timestamp: 2025-05-16T10:44:58Z
Top Bid: ['103704.6', '1269.21'] | Top Ask: ['103704.7', '1807.86']
----------------------------------------
Timestamp: 2025-05-16T10:44:58Z
Top Bid: ['103704.6', '1269.21'] | Top Ask: ['103704.7', '1807.94']
----------------------------------------
Timestamp: 2025-05-16T10:44:58Z
Top Bid: ['103704.6', '1275.63'] | Top Ask: ['103704.7', '1807.94']
----------------------------------------
Timestamp: 2025-05-16T10:44:58Z
Top Bid: ['103704.6', '1332.21'] | Top

In [21]:
pip install streamlit websocket-client


Timestamp: 2025-05-16T10:46:04Z
Top Bid: ['103732.5', '1067.9'] | Top Ask: ['103732.6', '889.35']
----------------------------------------
Timestamp: 2025-05-16T10:46:04Z
Top Bid: ['103732.5', '1067.9'] | Top Ask: ['103732.6', '892.98']
----------------------------------------
Timestamp: 2025-05-16T10:46:04Z
Top Bid: ['103732.5', '1067.9'] | Top Ask: ['103732.6', '892.98']
----------------------------------------
Timestamp: 2025-05-16T10:46:04Z
Top Bid: ['103732.5', '1069.9'] | Top Ask: ['103732.6', '892.1']
----------------------------------------
Timestamp: 2025-05-16T10:46:04Z
Top Bid: ['103732.5', '1069.91'] | Top Ask: ['103732.6', '892.1']
----------------------------------------
Timestamp: 2025-05-16T10:46:04Z
Top Bid: ['103732.5', '1069.04'] | Top Ask: ['103732.6', '892.1']
----------------------------------------
Timestamp: 2025-05-16T10:46:04Z
Top Bid: ['103732.5', '1069.04'] | Top Ask: ['103732.6', '891.95']
----------------------------------------
Timestamp: 2025-05-16T10:46

In [23]:
!streamlit run app.py

Timestamp: 2025-05-16T10:46:20Z
Top Bid: ['103708.3', '274.65'] | Top Ask: ['103708.4', '1613.18']
----------------------------------------
Timestamp: 2025-05-16T10:46:20Z
Top Bid: ['103708.3', '194.75'] | Top Ask: ['103708.4', '1693.2']
----------------------------------------
Timestamp: 2025-05-16T10:46:20Z
Top Bid: ['103702.8', '247.08'] | Top Ask: ['103702.9', '1884.2']
----------------------------------------
Timestamp: 2025-05-16T10:46:20Z
Top Bid: ['103702.8', '250.71'] | Top Ask: ['103702.9', '1823.84']
----------------------------------------
Timestamp: 2025-05-16T10:46:21Z
Top Bid: ['103700', '821.53'] | Top Ask: ['103700.1', '2146']
----------------------------------------
Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py


In [25]:
# Install necessary libraries
!pip install streamlit websocket-client streamlit-autorefresh

import streamlit as st
import websocket
import json
import threading
import time
from streamlit_autorefresh import st_autorefresh

# Use session state to store the latest orderbook data
if 'orderbook_data' not in st.session_state:
    st.session_state.orderbook_data = None

def on_message(ws, message):
    data = json.loads(message)
    # Store data in session state
    st.session_state.orderbook_data = data
    # Note: Updating session state from a background thread
    # won't instantly trigger a rerun. Streamlit will pick up
    # the change on the next natural rerun (e.g., widget interaction)
    # or via an auto-refresh mechanism.

def on_error(ws, error):
    # Check if Streamlit is running before calling st.error
    if st._is_running_with_streamlit:
        st.error(f"WebSocket error: {error}")
    else:
        print(f"WebSocket error: {error}")


def on_close(ws, close_status_code, close_msg):
     # Check if Streamlit is running before calling st.warning
    if st._is_running_with_streamlit:
        st.warning("WebSocket connection closed")
    else:
        print("WebSocket connection closed")

def on_open(ws):
    # Check if Streamlit is running before calling st.write
    if st._is_running_with_streamlit:
        st.write("WebSocket connection opened")
    else:
        print("WebSocket connection opened")


def run_ws():
    url = "wss://ws.gomarket-cpp.goquant.io/ws/l2-orderbook/okx/BTC-USDT-SWAP"
    ws = websocket.WebSocketApp(
        url,
        on_open=on_open,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close,
    )
    # run_forever() is blocking, so this thread will stay alive
    ws.run_forever()

def main():
    st.title("Real-time OKX BTC-USDT Orderbook")

    # Start websocket in background thread once
    if 'ws_thread' not in st.session_state:
        ws_thread = threading.Thread(target=run_ws, daemon=True)
        ws_thread.start()
        st.session_state.ws_thread = ws_thread

    # Optional: Add auto-refresh mechanism
    # Refresh every 1000 milliseconds (1 second)
    st_autorefresh(interval=1000, key="data_refresh")


    # UI elements for display - these will update when Streamlit reruns
    st.header("Top Bid and Ask Prices")

    # Display data from session state if available
    if st.session_state.orderbook_data:
        data = st.session_state.orderbook_data
        bids = data.get('bids', [])
        asks = data.get('asks', [])
        timestamp = data.get('timestamp', '')

        st.write(f"Last update: {timestamp}")
        if bids and asks:
            st.write(f"Top Bid Price: {bids[0][0]}, Quantity: {bids[0][1]}")
            st.write(f"Top Ask Price: {asks[0][0]}, Quantity: {asks[0][1]}")
        else:
            st.write("Waiting for orderbook data...")
    else:
        st.write("Waiting for initial orderbook data...")


if __name__ == "__main__":
    # Streamlit apps are executed from top to bottom on each interaction/rerun.
    # The main function is called by the 'streamlit run' command.
    # The WebSocket thread runs in the background and updates st.session_state.
    # The st_autorefresh component triggers reruns periodically.
    main()

Timestamp: 2025-05-16T10:47:00Z
Top Bid: ['103770.4', '964.61'] | Top Ask: ['103710.1', '16.67']
----------------------------------------
Timestamp: 2025-05-16T10:47:00Z
Top Bid: ['103770.4', '1058.61'] | Top Ask: ['103710.1', '16.67']
----------------------------------------
Timestamp: 2025-05-16T10:47:00Z
Top Bid: ['103770.4', '1178.41'] | Top Ask: ['103710.1', '16.67']
----------------------------------------
Timestamp: 2025-05-16T10:47:00Z
Top Bid: ['103770.4', '1217.34'] | Top Ask: ['103710.1', '16.67']
----------------------------------------
Timestamp: 2025-05-16T10:47:00Z
Top Bid: ['103770.4', '1217.39'] | Top Ask: ['103710.1', '16.67']
----------------------------------------
Timestamp: 2025-05-16T10:47:01Z
Top Bid: ['103770.4', '1178.09'] | Top Ask: ['103710.1', '16.67']
----------------------------------------
Timestamp: 2025-05-16T10:47:01Z
Top Bid: ['103770.4', '1218.09'] | Top Ask: ['103710.1', '16.67']
----------------------------------------
Timestamp: 2025-05-16T10:47:

2025-05-16 10:47:03.294 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 10:47:03.295 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 10:47:03.296 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 10:47:03.298 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 10:47:03.300 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 10:47:03.301 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 10:47:03.302 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 10:47:03.303 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar